In [8]:
import torch
import model
from torchsummary import summary
mod = model.get_RepVgg_func_by_name('RepVgg-A0')().cuda()

x = torch.randn((1,3,224,224)).cuda()
mod = model.repVgg_model_convert(mod)
print(summary(mod, input_size=(3,224,224)))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 48, 112, 112]           1,344
          Identity-2         [-1, 48, 112, 112]               0
              ReLU-3         [-1, 48, 112, 112]               0
       RepVggblock-4         [-1, 48, 112, 112]               0
            Conv2d-5           [-1, 48, 56, 56]          20,784
          Identity-6           [-1, 48, 56, 56]               0
              ReLU-7           [-1, 48, 56, 56]               0
       RepVggblock-8           [-1, 48, 56, 56]               0
            Conv2d-9           [-1, 48, 56, 56]          20,784
         Identity-10           [-1, 48, 56, 56]               0
             ReLU-11           [-1, 48, 56, 56]               0
      RepVggblock-12           [-1, 48, 56, 56]               0
           Conv2d-13           [-1, 96, 28, 28]          41,568
         Identity-14           [-1, 96,

In [1]:
import torch
import torchvision
import numpy as np
import os
import model
import wandb
import argparse
parse = argparse.ArgumentParser(description='input parameter of trian')

parse.add_argument('--batch-size', help = 'the size of training datesets', default=8,type=int)
parse.add_argument('--lr', help = 'lr', default=1e-3,type=float)
parse.add_argument('--epochs', help = 'nums of train', default=50)
parse.add_argument('--wandb', help = 'use or not use wandb', default=True, type=bool)
parse.add_argument('--yaml', help = 'yaml file path, edit', type = str)
args = parse.parse_args()

batch_size = args.batch_size
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
'''
data
'''
train_root = "D:/deep_learing_work/data/cif10/train/raw-img"
test_root = "D:/deep_learing_work/data/cif10/test/raw-img"
transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean = [0.485, 0.456,0.406],std = [0.229, 0.224, 0.225])
])
data_iter = torchvision.datasets.ImageFolder(root=train_root, transform=transform)
data_load = torch.utils.data.DataLoader(data_iter, batch_size = args.batch_size, shuffle = True)

test_iter = torchvision.datasets.ImageFolder(root=test_root, transform=transform)
test_load = torch.utils.data.DataLoader(test_iter, batch_size = args.batch_size, shuffle = False)

lossf = torch.nn.CrossEntropyLoss()
wandb.init(entity='wtj', project='Repvgg object classifier')
wandb.config = {
    'lr':args.lr,
    'epochs':args.epochs, 
    'batch_size':args.batch_size
}

modelRepVgg = model.get_RepVgg_func_by_name('RepVgg-A0')(num_classes = 10).to(device)
modelRepVgg.train()
optimizer = torch.optim.SGD(modelRepVgg.parameters(), lr = args.lr)

for eopch in range(args.epochs):
    loss_arr = []
    for x_cpu, y_cpu in data_load:
        optimizer.zero_grad()
        x = x_cpu.to(device)
        y = y_cpu.to(device)
        y_p = modelRepVgg(x)
        train_loss = lossf(y_p, y).sum()
        train_loss.backward()
        loss_arr.append(train_loss.detach().cpu())
        optimizer.step()
    modelRepVgg.eval()
    with torch.no_grad():
        train_mean_loss = torch.tensor(loss_arr)
        correct = 0
        total = 0
        for x, y in test_load:
            x, y = x.to(device),y.to(device)
            y_p = modelRepVgg(x)
            y_p = y_p.cpu()
            y_p = torch.argmax(y_p, dim = 1)
            label = y.cpu()
            correct +=  (y_p == label).sum()
            total += args.batch_size
        test_acc = correct / total
        print(test_acc, train_mean_loss)
        wandb.log({'train_loss':train_mean_loss,'test_acc:':test_acc})    
    modelRepVgg.train()
torch.save(modelRepVgg, './RepVgg.pt')
    



usage: ipykernel_launcher.py [-h] [--batch-size BATCH_SIZE] [--lr LR]
                             [--epochs EPOCHS] [--wandb WANDB] [--yaml YAML]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9004 --control=9002 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"eedaf893-6b27-447b-b829-b54a3f751750" --shell=9003 --transport="tcp" --iopub=9005 --f=C:\Users\10194\AppData\Local\Temp\tmp-734079ixIampGkL3.json


SystemExit: 2

C:\Users\10194\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
